In [ ]:
import pandas as pd
import numpy as np
import tensorqtl
from tensorqtl import genotypeio, cis, trans
import matplotlib.pyplot as plt

# define paths to data
plink_prefix_path = 'swath-ms.bgen'
expression_bed = 'swath-ms.expression.bed.gz'
covariates_file = 'swath-ms.covar'
prefix = 'swath-ms'

# load phenotypes and covariates
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T

# PLINK reader for genotypes
pr = genotypeio.PlinkReader(plink_prefix_path)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]


Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]

### *cis*- and *trans*-QTL mapping with tensorQTL

This notebook provides examples for running *cis*- and *trans*-QTL mapping with tensorQTL, using open-access data from the [GEUVADIS](https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/) project.

#### Requirements
An environment configured with a GPU and ~50GB of memory.

#### Test dataset

*Note: these files are provided for testing/benchmarking purposes only. They do not constitute an official release from the GEUVADIS project, and no quality-control was applied.*

Genotypes in PLINK and VCF format, and normalized expression data are available [here](https://personal.broadinstitute.org/francois/geuvadis/).

Alternatively, to download the files required for these examples, uncomment and run the cell below.

In [ ]:
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.nodup.bed
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.nodup.bim
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.GRCh38.20170504.maf01.filtered.nodup.fam   
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.covariates.txt
# !wget https://personal.broadinstitute.org/francois/geuvadis/GEUVADIS.445_samples.expression.bed.gz

### *cis*-QTL: nominal p-values for all variant-phenotype pairs

In [ ]:
# map all cis-associations (results for each chromosome are written to file)

# all genes
# cis.map_nominal(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df, prefix)

# genes on chr18
cis.map_nominal(genotype_df, variant_df, phenotype_df.loc[phenotype_pos_df['chr']=='chr18'],
                phenotype_pos_df.loc[phenotype_pos_df['chr']=='chr18'], covariates_df, prefix)

In [ ]:
# load results
pairs_df = pd.read_parquet('{}.cis_qtl_pairs.chr18.parquet'.format(prefix))
pairs_df.head()

### *cis*-QTL: empirical p-values for phenotypes

In [ ]:
# all genes
# cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df, phenotype_pos_df, covariates_df)

# genes on chr18
cis_df = cis.map_cis(genotype_df, variant_df, phenotype_df.loc[phenotype_pos_df['chr']=='chr18'],
                     phenotype_pos_df.loc[phenotype_pos_df['chr']=='chr18'], covariates_df)

In [ ]:
cis_df.head()

### *trans*-QTL mapping

In [ ]:
# run mapping
# to limit output size, only associations with p-value <= 1e-5 are returned
trans_df = trans.map_trans(genotype_df, phenotype_df, covariates_df, batch_size=20000,
                           return_sparse=True, pval_threshold=1e-5, maf_threshold=0.05)

In [ ]:
# remove cis-associations
trans_df = trans.filter_cis(trans_df, phenotype_pos_df.T.to_dict(), variant_df, window=5000000)

In [ ]:
trans_df.head()